# A toy example
# Convert to Julia file by jupytext --to jl ToyExample.ipynb

In [28]:
include("Pbox_nn.jl")
include("Sets_utils.jl")
include("Pforms.jl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


paf_focal_refinement_sauv (generic function with 1 method)

In [29]:
print_figures = false;

In [30]:
# Toy example: defining the network
W1 = [1 -1.0; 1.0 1.]
b = [0.0; 0.0]
W2 = [1 -1.0; 1.0 1.]
L1 = Layer(W1, b, ReLU())
L2 = Layer(W2, b, Id())
full_net = Network([L1; L2])

# input range
x = [interval(-2,2),interval(-1.,1.)]

2-element Vector{Interval{Float64}}:
 [-2, 2]
 [-1, 1]

In [31]:
# Running the example with 2 focal elements
#nb_discretization_steps = 2
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = uniform.(inf.(x),sup.(x)) 
# Printing the pbox
#pz = pbox_approximate_nnet(full_net,px,true) # independent inputs
#if print_figures
#    ProbabilityBoundsAnalysis.plot(px[1])
#    ProbabilityBoundsAnalysis.plot(pz[1])
#    ProbabilityBoundsAnalysis.plot(pz[2])
#end

In [32]:
function Compare_analyzes(px::Vector{pbox})

    println("DSI Analysis, unknown correlation on inputs")
    @time pz = pbox_approximate_nnet(full_net,px,false)
    print("pz[1] <= -2 : ",pz[1] <= -2,"\n")
    print("pz[1] >= 2 : ",pz[2] >= 2,"\n")


    println("DSI analysis, independent inputs")
    @time pz = pbox_approximate_nnet(full_net,px,true)
    print("pz[1] <= -2 : ",pz[1] <= -2,"\n")
    print("pz[1] >= 2 : ",pz[2] >= 2,"\n")


    println("Probabilistic zonotopes analysis, unknown correlation on inputs")
    @time affpz = affpbox_approximate_nnet(full_net,px,false)
    print("affpz[1] <= -2 : ",affpz[1] <= -2,"\n")
    print("affpz[1] >= 2 : ",affpz[2] >= 2,"\n")

    
    println("Probabilistic zonotope analysis, independent inputs")
    @time affpz = affpbox_approximate_nnet(full_net,px,true) # true for independent inputs
    print("affpz[1] <= -2 : ",affpz[1] <= -2,"\n")
    print("affpz[1] >= 2 : ",affpz[2] >= 2,"\n")

    #println("Zonotopic Dempster-Shafer analysis, independent inputs")
    #@time begin
    #    paf = generate_PAF(px)
    #    paf_res = paf_approximate_nnet(full_net, paf)
    #    dsztodsi = PAF_to_pbox(paf_res,true)
    #end
    #print("dszpz[1] <= -2 : ",dsztodsi[1] <= -2,"\n")
    #print("dszpz[1] >= 2 : ",dsztodsi[2] >= 2,"\n")

    println("Zonotopic Dempster-Shafer analysis, independent inputs")
    mat_spec_1 = [[1.0 0.0]
                  [0.0 -1.0]]
    rhs_spec_1 = [-2.0, -2.0]
    #@time paf_approximate_nnet_and_condition(full_net, px, mat_spec_1,rhs_spec_1,true) 
    println("Property conjunction")
    @time paf_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_1,rhs_spec_1,true) 

    #mat_spec_1stcondition = [1.0 0.0]
    ##rhs_spec_1stcondition = [-2.0]
    #mat_spec_2ndcondition = [0.0 -1.0]
    #rhs_spec_2ndcondition = [-2.0]
    #println("1st property")
    #@time paf_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_1stcondition,rhs_spec_1stcondition,true) 
    #println("2nd property")
    #@time paf_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_2ndcondition,rhs_spec_2ndcondition,true) 
end

Compare_analyzes (generic function with 1 method)

In [33]:
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = [(normal(interval(0,1),1)-0.5)/3.5902*2.,(normal(interval(0,1),1)/2.0-0.25)/1.7951]

for nb_discretization_steps in (2,10,100, 1000)
    println("Number of focal elements on the inputs = ",nb_discretization_steps)
    ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
    println("Uniform law")
    px = uniform.(inf.(x),sup.(x)) 
    Compare_analyzes(px)

    println("********************")
    println("Truncated Normal law")
    px = init_pbox_Normal(inf.(x),sup.(x),nb_discretization_steps,true) # truncated normal law
    Compare_analyzes(px)
    println("************************************************************************************************************************")
    println("************************************************************************************************************************")
end

Number of focal elements on the inputs = 2
Uniform law
DSI Analysis, unknown correlation on inputs
  0.046926 seconds (38.23 k allocations: 1.865 MiB, 96.80% compilation time: 36% of which was recompilation)
pz[1] <= -2 : [0, 0.5]
pz[1] >= 2 : [0, 1]
DSI analysis, independent inputs
  0.000834 seconds (6.48 k allocations: 243.984 KiB)
pz[1] <= -2 : [0, 0.5]
pz[1] >= 2 : [0, 1]
Probabilistic zonotopes analysis, unknown correlation on inputs
  0.059985 seconds (24.74 k allocations: 1.143 MiB, 64.18% gc time, 96.64% compilation time: 100% of which was recompilation)
affpz[1] <= -2 : [0, 0.5]
affpz[1] >= 2 : [0, 1]
Probabilistic zonotope analysis, independent inputs
  0.001048 seconds (6.89 k allocations: 228.828 KiB)
affpz[1] <= -2 : [0, 0.5]
affpz[1] >= 2 : [0, 1]
Zonotopic Dempster-Shafer analysis, independent inputs
Number of zonotopic focal elements=4
  0.075262 seconds (79.95 k allocations: 4.338 MiB, 99.33% compilation time)
dszpz[1] <= -2 : [0, 0.25]
dszpz[1] >= 2 : [0, 0.5]
Zonoto